In [143]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

In [88]:
links = pd.read_csv('C:\\Users\\Андрей Клычев\\ml-latest-small\\links.csv')
movies = pd.read_csv('C:\\Users\\Андрей Клычев\\ml-latest-small\\movies.csv')
ratings = pd.read_csv('C:\\Users\\Андрей Клычев\\ml-latest-small\\ratings.csv')
tags = pd.read_csv('C:\\Users\\Андрей Клычев\\ml-latest-small\\tags.csv')

In [89]:
tags['date'] = tags['timestamp'].apply(lambda t: datetime.fromtimestamp(t))
ratings['date'] = ratings['timestamp'].apply(lambda t: datetime.fromtimestamp(t))

In [90]:
import re
def divide(row):
   
    row['year'] = re.findall('\d{4}', row['title'])
   
    return row
    
movies = movies.apply(divide, axis=1)
m = []
for _ in movies.year:
    try:
        m.append(int(_[0]))
    except:
        m.append(0)


movies.year = m


In [91]:
movies = movies.merge(ratings, how='left', on='movieId')

Исходя из жанра и даты определим недостающие рейтинги  
Количество жанров достигать может 10 штук  
И все же линки с те
гами тоже нужно использовать  и восполнять все эти пропуски регрессией

In [92]:
movies = movies.drop(['timestamp'], axis=1)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import NearestNeighbors

In [93]:
def div(row):
    
    return row['genres'].replace('|', ' ')

def diff(row):
    return row['date'].year - row['year']

d = movies.apply(diff, axis=1)
movies.date = d


In [94]:
movies = movies.dropna()
g = movies.apply(div, axis=1)
movies.genres = g

In [99]:
movies = movies.merge(tags, how='left', on='movieId')
movies = movies.dropna()

In [98]:
def sum_str(row):
    row['words'] = row['genres'] + ' ' + row['tag']
    return row
movies = movies.apply(sum_str, axis=1)
movies

,movieId,title,genres,year,userId_x,rating,date_x,userId_y,tag,timestamp,date_y,words
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,1.0,4.0,5.0,336.0,pixar,1.139046e+09,2006-02-04 12:36:04,Adventure Animation Children Comedy Fantasy pixar
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,1.0,4.0,5.0,474.0,pixar,1.137207e+09,2006-01-14 05:47:05,Adventure Animation Children Comedy Fantasy pixar
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,1.0,4.0,5.0,567.0,fun,1.525286e+09,2018-05-02 21:33:33,Adventure Animation Children Comedy Fantasy fun
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,5.0,4.0,1.0,336.0,pixar,1.139046e+09,2006-02-04 12:36:04,Adventure Animation Children Comedy Fantasy pixar
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1995,5.0,4.0,1.0,474.0,pixar,1.137207e+09,2006-01-14 05:47:05,Adventure Animation Children Comedy Fantasy pixar
...,...,...,...,...,...,...,...,...,...,...,...,...
285724,187595,Solo: A Star Wars Story (2018),Action Adventure Children Sci-Fi,2018,586.0,5.0,0.0,62.0,star wars,1.528935e+09,2018-06-14 03:02:32,Action Adventure Children Sci-Fi star wars
285749,193565,Gintama: The Movie (2010),Action Animation Comedy Sci-Fi,2010,184.0,3.5,8.0,184.0,anime,1.537099e+09,2018-09-16 14:49:42,Action Animation Comedy Sci-Fi anime
285750,193565,Gintama: The Movie (2010),Action Animation Comedy Sci-Fi,2010,184.0,3.5,8.0,184.0,comedy,1.537099e+09,2018-09-16 14:49:47,Action Animation Comedy Sci-Fi comedy
285751,193565,Gintama: The Movie (2010),Action Animation Comedy Sci-Fi,2010,184.0,3.5,8.0,184.0,gintama,1.537099e+09,2018-09-16 14:50:03,Action Animation Comedy Sci-Fi gintama


In [100]:
cntV = CountVectorizer()
vect_g = cntV.fit_transform(movies.words)
tfidf = TfidfTransformer()
vect_tf = tfidf.fit_transform(vect_g)

In [105]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(vect_tf.toarray(), movies.rating, test_size=0.3)

In [107]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [108]:
y_pred = model.predict(x_test)

In [166]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)

36245654405.75947

In [154]:
m = movies.groupby('title').mean().rating
med = movies.groupby('title').median().rating
std = movies.groupby('title').std().rating

In [162]:
def merging(row):
    for _ in range(len(m.index)):
        if m.index[_] == row.title:
            row['mean_rating'] = m[m.index == row.title][0]
            row['std_rating'] = std[std.index == row.title][0]
            row['median_rating'] = med[med.index == row.title][0]
    return row

In [164]:
# movies = movies.apply(merging, axis=1)


# Не могу взять для целей обучения Средние, медианные, значения, по-скольку невозможно внести впоследствии эти значения в таблицу  
# Функция 'merging' работает бесконечно долго, точнее вообще не выполняется =(